In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
from re import search

# Import de la base de données nettoyée

In [2]:
df_airbnb = pd.read_csv('data_cleanedV4.csv', index_col=0)

In [3]:
pd.set_option('display.max_columns', None)
df_airbnb.head()

,ID,Name,Summary,Space,Neighborhood Overview,Notes,Transit,Interaction,Host Since,Host Verifications,Neighbourhood Cleansed,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Cancellation Policy,Reviews per Month,Geolocation,Features
0,5380461,"Roland Garros, Appart. 2 p. avec balcon, Auteuil.","Appart. ascenc. 1 ch. avec lit double, cuisine...",L'appartement est situé dans le quartier résid...,"J'apprécie le côté résidentiel du quartier, ca...",Nombreux commerces et restaurants à proximité.,Vous pourrez accéder à l'appartement par les l...,Je souhaite louer mon appartement à des person...,2015-02-17,"email,phone,reviews",Passy,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"TV,Internet,Wireless Internet,Kitchen,Elevator...",50.0,500.0,30.0,1,0,4 days ago,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,flexible,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact"
1,12994171,APARTMENT 100 M2,location trocadero Passy very well located ans...,la situation et la tranquillité ainsi que l'ac...,situation à 5 min à pied de la tour Eiffel et ...,NaN,métro à 3 min à pied,si vous avez des besoins particuliers ...cours...,2016-05-12,"email,phone,reviews",Passy,Apartment,Entire home/apt,5,2.0,2.0,2.0,Real Bed,"TV,Cable TV,Internet,Wireless Internet,Wheelch...",250.0,750.0,90.0,4,15,today,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,strict,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact"
2,16270986,Appartement familial église d'auteuil,"Mon logement est proche de Beaugrenelle Paris,...",NaN,NaN,NaN,NaN,NaN,2016-10-25,"email,phone,facebook,reviews",Passy,Apartment,Entire home/apt,7,1.5,2.0,5.0,Real Bed,"TV,Wireless Internet,Air conditioning,Kitchen,...",165.0,200.0,30.0,1,0,today,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,flexible,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant..."
3,16267918,Studio 25m2 16e arrondissement,Petit studio situé dans le 16 e arrondissement...,NaN,"Boulangerie, coiffeur, Franprix etc",NaN,Bus 72 et 22 arrêt victorien Sardou Météo lig...,NaN,2015-12-11,"email,phone,reviews",Passy,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"TV,Internet,Wireless Internet,Kitchen,Elevator...",35.0,94.0,0.0,1,0,4 months ago,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,moderate,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact"
4,8531724,Cosy and bright flat,"Nice flat, recently renovated, bright and cosy...",The flat is in the 4th floor of a secure build...,The area is very pleasant and chic. It is both...,The bedsheets and towels are included in the r...,The flat is very well located for transportati...,I will welcome you in the flat to give you the...,2014-01-28,"email,phone,reviews",Passy,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"TV,Internet,Wireless Internet,Kitchen,Elevator...",90.0,500.0,30.0,2,10,6 months ago,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,moderate,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact"


# Colonnes de texte

Suppression des colonnes Unnamed: 0, ID, Name, Summary, Space, Neighborhood Overview, Notes, Transit et Interaction qui contiennent beaucoup de texte et qui semblent difficelement recodable. 

In [226]:
# Suppression des colonnes 
df = df_airbnb.copy()
df.drop(columns=['Unnamed: 0','ID','Name','Summary','Space','Neighborhood Overview','Notes','Transit','Interaction'], inplace=True)

In [227]:
df.columns

Index(['Host Since', 'Host Verifications', 'Neighbourhood Cleansed',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Price', 'Security Deposit',
       'Cleaning Fee', 'Guests Included', 'Extra People', 'Calendar Updated',
       'Availability 60', 'Review Scores Rating', 'Review Scores Accuracy',
       'Review Scores Cleanliness', 'Review Scores Checkin',
       'Review Scores Communication', 'Review Scores Location',
       'Review Scores Value', 'Cancellation Policy', 'Reviews per Month',
       'Geolocation', 'Features'],
      dtype='object')

# Colonne 'Host Since'

Le but est de transformer les dates auxquelles l'hôte s'est inscrit, en une colonne donnant le nombre de jours écoulés depuis l'inscription de l'hôte  
<=> Depuis combien de jours l'hôte est-il inscrit ? 

In [228]:
df1 = df.copy()

In [229]:
# Création d'une liste qui soustraie la date d'aujourd'hui à la date à laquelle l'hôte s'est inscrit 
# Retourne le nombre jours correspondant pour chaque hôte, dans une liste 

def Host_Since_toint(date_inscription) -> int:
    """Renvoit l'ancienneté de l'hôte à la date du 2020/04/15"""
    
    today = datetime.strptime('2020/04/15',"%Y/%m/%d") 
    try:
        delta = today - date_inscription
        return delta.days
    except:
        return date_inscription

In [230]:
# Conversion de la colonne 'Host since' en date puis application de la fonction créé précedemment 
df1['Host Since'] =  pd.to_datetime(df1['Host Since']).apply(Host_Since_toint)

# Colonne 'Calendar Updated'

Calcul du nombre de jours depuis la dernière mise à jour du calendrier. Résultat : nombre de jours depuis lequel la dernière mise à jour a été effectuée

In [231]:
df2 = df1.copy()

In [232]:
def Calendar_Update_toint(sentence: str) -> int:
    """
    Renvoie le nombre de jours depuis la dernière mise à jour du calendrier
    """
    sentence = sentence.strip().lower()
    if sentence == "today":
        return 0
    if sentence == "yesterday":
        return 1
    try:
        number = int(sentence.split()[0])
    except ValueError:
        number = 1 if (sentence.split()[0] == "a") else None
    if "days" in sentence:
        return number
    if "week" in sentence:
        return number * 7
    if "month" in sentence:
        return number * 30
    else:
        return None

In [233]:
df2['Calendar Updated'] = df2['Calendar Updated'].apply(Calendar_Update_toint)

# Colonne 'Bed Type'

Transformation des modalités de la variable 'Bed Type'.  
La nouvelle variable renvoie 1 si la modalité correspond à "Real Bed" et 0 sinon. 

In [234]:
df3 = df2.copy()

In [235]:
def Bed_type_toint(bed: str) -> int:
    """
    Binarisation sur la base d'un vrai lit ou non 
    """
    try:
        dico = {
            "Airbed" : 0,
            "Couch" : 0,
            "Futon" : 0,
            "Pull-out Sofa": 0,
            "Real Bed": 1
        }
    
        return dico[bed.strip()]
    except:
        return bed

In [236]:
df3['Bed Type'] = df3['Bed Type'].apply(Bed_type_toint)

In [237]:
df3['Bed Type'].value_counts()

1    35644
0     4384
Name: Bed Type, dtype: int64

# Colonne 'Cancellation Policy'

Binarisation de chacune des modalités de cette colonne

In [238]:
df4 = df3.copy()

In [239]:
df4['Cancellation Policy'].value_counts()

strict             15623
moderate           13030
flexible           11371
super_strict_30        4
Name: Cancellation Policy, dtype: int64

In [240]:
def Policy_encoding (df, modalite) : 
    liste = []

    for i in range(len(df)): 
        if modalite in str(df[i]):
            #Si la modalité est présente, on lui attribut le code 1 
            liste.append(int(1))
        else: 
            #Sinon on lui attribut le code 0
            liste.append(int(0))

    df_liste = pd.DataFrame(liste)
    return df_liste

In [241]:
df4['strict_cancel_policy'] = Policy_encoding(df4['Cancellation Policy'],'strict')

In [242]:
df4['moderate_cancel_policy'] = Policy_encoding(df4['Cancellation Policy'],'moderate')

In [243]:
df4['flexible_cancel_policy'] = Policy_encoding(df4['Cancellation Policy'],'flexible')

In [244]:
df4['super_strict_cancel_policy'] = Policy_encoding(df4['Cancellation Policy'],'super_strict_30')

In [245]:
df4.drop(columns=['Cancellation Policy'], inplace=True)

In [246]:
df4.head()

,Host Since,Host Verifications,Neighbourhood Cleansed,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Geolocation,Features,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy
0,1884.0,"email,phone,reviews",Passy,Apartment,Entire home/apt,2,1.0,1.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",50.0,500.0,30.0,1,0,4.0,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact",0,0,1,0
1,1434.0,"email,phone,reviews",Passy,Apartment,Entire home/apt,5,2.0,2.0,2.0,1,"TV,Cable TV,Internet,Wireless Internet,Wheelch...",250.0,750.0,90.0,4,15,0.0,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact",1,0,0,0
2,1268.0,"email,phone,facebook,reviews",Passy,Apartment,Entire home/apt,7,1.5,2.0,5.0,1,"TV,Wireless Internet,Air conditioning,Kitchen,...",165.0,200.0,30.0,1,0,0.0,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant...",0,0,1,0
3,1587.0,"email,phone,reviews",Passy,Apartment,Entire home/apt,2,1.0,0.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",35.0,94.0,0.0,1,0,120.0,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact",0,1,0,0
4,2269.0,"email,phone,reviews",Passy,Apartment,Entire home/apt,4,1.0,1.0,2.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",90.0,500.0,30.0,2,10,180.0,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact",0,1,0,0


# Colonne 'Room Type'

Binarisation de chacune des modalités de la colonne 'Room Type'. 

In [247]:
df5 = df4.copy()

In [248]:
df5['Room Type'].value_counts()

Entire home/apt    34727
Private room        4924
Shared room          377
Name: Room Type, dtype: int64

In [249]:
def Room_type_encoding (df, modalite) : 
    
    liste = []

    for i in range(len(df)): 
        if modalite in str(df[i]):
            #Si la modalité est présente, on lui attribut le code 1 
            liste.append(int(1))
        else: 
            #Sinon on lui attribut le code 0
            liste.append(int(0))

    df_liste = pd.DataFrame(liste)
    return df_liste

In [250]:
df5['entire_home'] = Room_type_encoding(df5['Room Type'],'Entire home/apt')

In [251]:
df5['private_room'] = Room_type_encoding(df5['Room Type'],'Private room')

In [252]:
df5['shared_room'] = Room_type_encoding(df5['Room Type'],'Shared room')

In [253]:
df5.drop(columns=['Room Type'], inplace=True)

In [254]:
df5.head()

,Host Since,Host Verifications,Neighbourhood Cleansed,Property Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Geolocation,Features,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room
0,1884.0,"email,phone,reviews",Passy,Apartment,2,1.0,1.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",50.0,500.0,30.0,1,0,4.0,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact",0,0,1,0,1,0,0
1,1434.0,"email,phone,reviews",Passy,Apartment,5,2.0,2.0,2.0,1,"TV,Cable TV,Internet,Wireless Internet,Wheelch...",250.0,750.0,90.0,4,15,0.0,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact",1,0,0,0,1,0,0
2,1268.0,"email,phone,facebook,reviews",Passy,Apartment,7,1.5,2.0,5.0,1,"TV,Wireless Internet,Air conditioning,Kitchen,...",165.0,200.0,30.0,1,0,0.0,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant...",0,0,1,0,1,0,0
3,1587.0,"email,phone,reviews",Passy,Apartment,2,1.0,0.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",35.0,94.0,0.0,1,0,120.0,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0
4,2269.0,"email,phone,reviews",Passy,Apartment,4,1.0,1.0,2.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",90.0,500.0,30.0,2,10,180.0,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0


# Colonne 'Property Type'

Traitement de la colonne 'Property Type'. Regroupement des types de logement en 5 classes, puis binarisation de chaque classe :   
Other = modalités 'Boat' + 'Boutique hotel' + 'Bungalow' + 'Cabin' + 'Camper/RV' + 'Cave' +'Earth House' + 'Igloo'+ 'Other' + 'Timeshare' + 'Tipi' + 'Treehouse'    
Hostel = modalités 'Dorm' + 'Hostel'  
Bed&Breakfast = modalités 'Bed & Breakfast' + 'Guesthouse'   
Apartment = modalités 'Condominium' + 'Apartment' + 'Serviced apartment'   
House = modalités 'House' + 'Loft'+ 'Townhouse' + 'Villa'   

In [255]:
df6 = df5.copy()

In [256]:
df6['Property Type'].replace('Earth House','Earth_house', inplace=True)

In [257]:
hostel = ['Dorm','Hostel']
bed_breakfast = ['Bed & Breakfast','Guesthouse']
apartment = ['Condominium','Apartment','Serviced apartment']
house = ['House','Loft','Townhouse','Villa']
other = ['Boat','Boutique hotel','Bungalow','Cabin','Camper/RV','Cave','Earth_house','Igloo','Other','Timeshare','Tipi','Treehouse']

In [258]:
def Property_encoder(df,classe):

    property_type = []

    for i in range(len(df)): 
        
        if any(item in df[i] for item in classe): 
            property_type.append(int(1))
        else : 
            property_type.append(int(0))

    df_property = pd.DataFrame(property_type)
    return df_property

In [259]:
df6['property_hostel'] = Property_encoder(df6['Property Type'],hostel)

In [260]:
df6['property_bed_breakfast'] = Property_encoder(df6['Property Type'],bed_breakfast)

In [261]:
df6['property_apartment'] = Property_encoder(df6['Property Type'],apartment)

In [262]:
df6['property_house'] = Property_encoder(df6['Property Type'],house)

In [263]:
df6['other_property'] = Property_encoder(df6['Property Type'],other)

In [264]:
df6.drop(columns=['Property Type'], inplace=True)

In [265]:
df6.head()

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Geolocation,Features,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property
0,1884.0,"email,phone,reviews",Passy,2,1.0,1.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",50.0,500.0,30.0,1,0,4.0,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact",0,0,1,0,1,0,0,0,0,1,0,0
1,1434.0,"email,phone,reviews",Passy,5,2.0,2.0,2.0,1,"TV,Cable TV,Internet,Wireless Internet,Wheelch...",250.0,750.0,90.0,4,15,0.0,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact",1,0,0,0,1,0,0,0,0,1,0,0
2,1268.0,"email,phone,facebook,reviews",Passy,7,1.5,2.0,5.0,1,"TV,Wireless Internet,Air conditioning,Kitchen,...",165.0,200.0,30.0,1,0,0.0,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant...",0,0,1,0,1,0,0,0,0,1,0,0
3,1587.0,"email,phone,reviews",Passy,2,1.0,0.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",35.0,94.0,0.0,1,0,120.0,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0
4,2269.0,"email,phone,reviews",Passy,4,1.0,1.0,2.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",90.0,500.0,30.0,2,10,180.0,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0


# Colonne 'Amenities'

De cette colonne, nous allons récuperer deux informations :  
1) Binariser les Amenities les plus fréquemment mentionnées dans les annonces de notre dataset  
2) Le nombre d'Amenities mentionnées dans l'annonce  

In [266]:
df7 = df6.copy()

## Binarisation des Amenities les plus importants

### Définition des variables les plus importantes

In [267]:
# Conversion des modalités de la colonne Amenities en liste de liste 
col_list = df7['Amenities'].str.split(',').values.tolist()

In [268]:
col_list[0:1]

[['TV',
  'Internet',
  'Wireless Internet',
  'Kitchen',
  'Elevator in building',
  'Heating',
  'Washer',
  'Smoke detector',
  'Essentials',
  'Hangers',
  'Hair dryer',
  'Iron',
  'translation missing: en.hosting_amenity_49',
  'translation missing: en.hosting_amenity_50']]

In [269]:
# Fonction pour transformer la liste de liste en liste 
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [270]:
flat_list = flatten(col_list)

In [271]:
flat_list[0:20]

['TV',
 'Internet',
 'Wireless Internet',
 'Kitchen',
 'Elevator in building',
 'Heating',
 'Washer',
 'Smoke detector',
 'Essentials',
 'Hangers',
 'Hair dryer',
 'Iron',
 'translation missing: en.hosting_amenity_49',
 'translation missing: en.hosting_amenity_50',
 'TV',
 'Cable TV',
 'Internet',
 'Wireless Internet',
 'Wheelchair accessible',
 'Kitchen']

In [272]:
df_amenities = pd.DataFrame(flat_list, columns=['Amenities'])

In [273]:
df_count = pd.DataFrame(df_amenities['Amenities'].value_counts()).reset_index()
df_count.columns = ['Amenities','Count']

In [274]:
# Les 20 Amenities les plus cités dans les annonces
df_count.iloc[0:40,:]

,Amenities,Count
0,Heating,38684
1,Kitchen,38564
2,Wireless Internet,38361
3,Essentials,34846
4,Washer,30778
5,Internet,28895
6,TV,26729
7,Smoke detector,24667
8,Family/kid friendly,22774
9,Hair dryer,22457


Nous allons maintenant binariser les équipements qui paraissent le plus pertinent d'un point de vue métier parmi les 35 les plus cités dans les annonces.  
=> Heating, Kitchen, Wireless Internet, Essentials, Washer, TV, Family/kid friendly, Hair dryer, Iron, Elevator in building, Dryer, 24-hour check-in, Smoking allowed, Breakfast, Pets allowed, Wheelchair accessible, Air conditioning, Free parking on premises

### Binarisation des Amenities les plus importants

In [275]:
def Amenities_encoder(df,amenities):

    """
    Fonction permettant de binariser un amenities 
    df: df7['Amenities']
    amenities : l'équipement que l'on souhaite binariser 
    """
    amenities_list = []

    for i in range(len(df)): 
        if amenities in str(df[i]):
            amenities_list.append(int(1))
        else : 
            amenities_list.append(int(0))

    df_amenities = pd.DataFrame(amenities_list)
    return df_amenities

In [276]:
def Get_variables(df): 
    
    """
    Fonction qui permet d'ajouter au DataFrame d'entré chaque amenities binarisé
    df : df7
    """
    main_amenities = ['Heating', 'Kitchen', 'Wireless Internet', 'Essentials', 'Washer', 'TV',
                      'Family/kid friendly', 'Hair dryer', 'Iron', 'Elevator in building', 'Dryer', 
                      '24-hour check-in', 'Smoking allowed', 'Breakfast', 'Pets allowed', 'Wheelchair accessible', 
                      'Air conditioning', 'Free parking on premises']
    for i in main_amenities:
        df[i] = Amenities_encoder(df['Amenities'],i)
    
    return df 

In [277]:
Get_variables(df7)

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Geolocation,Features,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property,Heating,Kitchen,Wireless Internet,Essentials,Washer,TV,Family/kid friendly,Hair dryer,Iron,Elevator in building,Dryer,24-hour check-in,Smoking allowed,Breakfast,Pets allowed,Wheelchair accessible,Air conditioning,Free parking on premises
0,1884.0,"email,phone,reviews",Passy,2,1.0,1.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",50.0,500.0,30.0,1,0,4.0,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0
1,1434.0,"email,phone,reviews",Passy,5,2.0,2.0,2.0,1,"TV,Cable TV,Internet,Wireless Internet,Wheelch...",250.0,750.0,90.0,4,15,0.0,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0
2,1268.0,"email,phone,facebook,reviews",Passy,7,1.5,2.0,5.0,1,"TV,Wireless Internet,Air conditioning,Kitchen,...",165.0,200.0,30.0,1,0,0.0,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant...",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0
3,1587.0,"email,phone,reviews",Passy,2,1.0,0.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",35.0,94.0,0.0,1,0,120.0,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
4,2269.0,"email,phone,reviews",Passy,4,1.0,1.0,2.0,1,"TV,Internet,Wireless Internet,Kitchen,Elevator...",90.0,500.0,30.0,2,10,180.0,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40023,2072.0,"email,phone,facebook,reviews",Ménilmontant,2,1.0,1.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Free par...",40.0,300.0,19.0,1,5,7.0,10,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.71,"48.8619682733,2.40198612937","Host Has Profile Pic,Is Location Exact,Instant...",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,0,1
40024,1983.0,"email,phone,reviews,jumio",Ménilmontant,2,1.0,0.0,2.0,1,"TV,Wireless Internet,Kitchen,Smoking allowed,E...",26.0,200.0,25.0,1,0,240.0,0,80.0,10.0,10.0,10.0,10.0,10.0,10.0,0.13,"48.8664429191,2.39643395508","Host Has Profile Pic,Host Identity Verified,Is...",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0
40025,1863.0,"email,phone,reviews",Ménilmontant,2,1.0,1.0,1.0,0,"Internet,Wireless Internet,Kitchen,Smoking all...",50.0,0.0,0.0,1,0,7.0,10,100.0,10.0,10.0,9.0,10.0,10.0,10.0,0.55,"48.8663673103,2.39473441637","Host Has Profile Pic,Is Location Exact",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0
40026,2674.0,"email,phone,reviews,jumio,government_id",Ménilmontant,2,1.0,1.0,1.0,1,"TV,Internet,Wireless Internet,Kitchen,Smoking ...",40.0,200.0,0.0,1,30,0.0,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.47,"48.8680706349,2.39800286126","Host Has Profile Pic,Host Identity Verified,Is...",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0


## Calcul du nombre d'Amenities

In [278]:
def Count(sentence : str) -> int:
    """Retourne le nombre d'Amenities"""
    try:
        return len(sentence.strip().split(','))
    except AttributeError:
        if isinstance(df.Amenities[0], np.int64):
            return sentence
        #si c'est un nan
        else:
            print()
            return 0

In [279]:
df7['Amenities'] =  df7['Amenities'].apply(Count)

# Colonne 'Host Verifications'

De cette colonne, nous allons récuperer deux informations :  
1) Binariser les documents prouvant l'identité de l'hôte les plus fréquemment mentionnées dans les annonces de notre dataset et qui semblent le plus pertinent  
2) Le nombre de documents prouvant l'identité de l'hôte vérifiés par Airbnb

In [280]:
df8 = df7.copy()

## Binarisation des documents vérifiés les plus importants

### Définition des variables les plus importantes

In [281]:
# Conversion des modalités de la colonne Amenities en liste de liste 
col_list2 = df8['Host Verifications'].str.split(',').values.tolist()

In [282]:
flat_list2 = flatten(col_list2)

In [283]:
df_verifications = pd.DataFrame(flat_list2, columns=['Verifications'])

In [284]:
df_count_verif = pd.DataFrame(df_verifications['Verifications'].value_counts()).reset_index()
df_count_verif.columns = ['Verifications','Count']

In [285]:
df_count_verif.head(20)

,Verifications,Count
0,reviews,39946
1,phone,39571
2,email,39078
3,jumio,23917
4,facebook,7173
5,government_id,6542
6,work_email,2627
7,manual_offline,1512
8,offline_government_id,1501
9,google,1208


Nous allons maintenant binariser les documents vérifiés prouvant l'identité qui paraissent le plus pertinent d'un point de vue métier parmi les 20 plus fréquents.  
=> reviews, phone, email, government_id

### Binarisation des Host Verifications les plus importants

In [286]:
def Verifications_encoder(df,verification):

    """
    Fonction permettant de binariser un type de document d'identité vérifié
    df: df8['Host Verifications']
    verification : Le document d'identité de l'hôte vérifié par Airbnb
    """
    verification_list = []

    for i in range(len(df)): 
        if verification in str(df[i]):
            verification_list.append(int(1))
        else : 
            verification_list.append(int(0))

    df_verification = pd.DataFrame(verification_list)
    return df_verification

In [287]:
def Get_verifications(df): 
    
    """
    Fonction qui permet d'ajouter au DataFrame d'entré chaque document vérifié binarisé
    df : df8
    """
    main_verifications = ['reviews', 'phone', 'email', 'government_id']
    for i in main_verifications:
        name = 'host'+ '_'+ i + '_' + 'verified'
        df[name] = Verifications_encoder(df['Host Verifications'],i)
    
    return df 

In [288]:
Get_verifications(df8)

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Geolocation,Features,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property,Heating,Kitchen,Wireless Internet,Essentials,Washer,TV,Family/kid friendly,Hair dryer,Iron,Elevator in building,Dryer,24-hour check-in,Smoking allowed,Breakfast,Pets allowed,Wheelchair accessible,Air conditioning,Free parking on premises,host_reviews_verified,host_phone_verified,host_email_verified,host_government_id_verified
0,1884.0,"email,phone,reviews",Passy,2,1.0,1.0,1.0,1,14,50.0,500.0,30.0,1,0,4.0,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0
1,1434.0,"email,phone,reviews",Passy,5,2.0,2.0,2.0,1,23,250.0,750.0,90.0,4,15,0.0,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,0
2,1268.0,"email,phone,facebook,reviews",Passy,7,1.5,2.0,5.0,1,19,165.0,200.0,30.0,1,0,0.0,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant...",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0
3,1587.0,"email,phone,reviews",Passy,2,1.0,0.0,1.0,1,13,35.0,94.0,0.0,1,0,120.0,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0
4,2269.0,"email,phone,reviews",Passy,4,1.0,1.0,2.0,1,15,90.0,500.0,30.0,2,10,180.0,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40023,2072.0,"email,phone,facebook,reviews",Ménilmontant,2,1.0,1.0,1.0,1,19,40.0,300.0,19.0,1,5,7.0,10,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.71,"48.8619682733,2.40198612937","Host Has Profile Pic,Is Location Exact,Instant...",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,0,1,1,1,1,0
40024,1983.0,"email,phone,reviews,jumio",Ménilmontant,2,1.0,0.0,2.0,1,15,26.0,200.0,25.0,1,0,240.0,0,80.0,10.0,10.0,10.0,10.0,10.0,10.0,0.13,"48.8664429191,2.39643395508","Host Has Profile Pic,Host Identity Verified,Is...",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0
40025,1863.0,"email,phone,reviews",Ménilmontant,2,1.0,1.0,1.0,0,15,50.0,0.0,0.0,1,0,7.0,10,100.0,10.0,10.0,9.0,10.0,10.0,10.0,0.55,"48.8663673103,2.39473441637","Host Has Profile Pic,Is Location Exact",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,1,1,1,0
40026,2674.0,"email,phone,reviews,jumio,government_id",Ménilmontant,2,1.0,1.0,1.0,1,10,40.0,200.0,0.0,1,30,0.0,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.47,"48.8680706349,2.39800286126","Host Has Profile Pic,Host Identity Verified,Is...",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1,1


## Calcul du nombre de Host verifications

In [289]:
df8['Host Verifications'] =  df8['Host Verifications'].apply(Count)

# Colonne 'Features'

A partir des modalités de la colonne 'Features', création de 5 nouvelles variables.  
Les 5 modalités qui semblent le plus pertinentes à inclure comme variable explicatives sont 'Host Is Superhost', 'Is Location Exact', 'Host Has Profile Pic', 'Host Identity Verified', 'Instant Bookable'. Donc nous créons ces 4 nouvelles variables : si l'observation i avait la modalité j, alors on met 1 dans la variable créée à partir de la modalité j et pour l'observation i (0 sinon). 

In [290]:
df9 = df8.copy()

In [291]:
def Features_encoder (df, modalite):
    """
    Fonction permettant de binariser un Feature de l'annonce important
    df: df9['Features']
    modalite : modalité transformée en variable binaire
    """
    list_binarisation = []

    for i in range(len(df)): 
        if modalite in str(df[i]):
            #Si l'annonce valide ce critère, on lui attribut le code 1 
            list_binarisation.append(int(1))
        else: 
            #Sinon on lui attribut le code 0
            list_binarisation.append(int(0))

    df_features = pd.DataFrame(list_binarisation)
    return df_features

In [292]:
def Get_features(df): 
    
    """
    Fonction qui permet d'ajouter au DataFrame d'entré chaque feature binarisé 
    df : df9
    """
    main_features = ['Host Is Superhost', 'Is Location Exact','Host Has Profile Pic', 
                     'Host Identity Verified', 'Instant Bookable']
    for i in main_features:
        df[i] = Features_encoder(df['Features'],i)
    return df 

In [293]:
Get_features(df9)

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Geolocation,Features,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property,Heating,Kitchen,Wireless Internet,Essentials,Washer,TV,Family/kid friendly,Hair dryer,Iron,Elevator in building,Dryer,24-hour check-in,Smoking allowed,Breakfast,Pets allowed,Wheelchair accessible,Air conditioning,Free parking on premises,host_reviews_verified,host_phone_verified,host_email_verified,host_government_id_verified,Host Is Superhost,Is Location Exact,Host Has Profile Pic,Host Identity Verified,Instant Bookable
0,1884.0,3,Passy,2,1.0,1.0,1.0,1,14,50.0,500.0,30.0,1,0,4.0,15,80.0,7.0,8.0,9.0,9.0,10.0,7.0,0.16,"48.8445603313,2.26336692547","Host Has Profile Pic,Is Location Exact",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0
1,1434.0,3,Passy,5,2.0,2.0,2.0,1,23,250.0,750.0,90.0,4,15,0.0,28,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.56,"48.8580714183,2.28022090415","Host Has Profile Pic,Is Location Exact",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,0,0,1,1,0,0
2,1268.0,4,Passy,7,1.5,2.0,5.0,1,19,165.0,200.0,30.0,1,0,0.0,51,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1.00,"48.8502032736,2.27083162889","Host Has Profile Pic,Is Location Exact,Instant...",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,1
3,1587.0,3,Passy,2,1.0,0.0,1.0,1,13,35.0,94.0,0.0,1,0,120.0,0,93.0,10.0,10.0,10.0,10.0,9.0,10.0,1.38,"48.8420130083,2.26885375663","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0
4,2269.0,3,Passy,4,1.0,1.0,2.0,1,15,90.0,500.0,30.0,2,10,180.0,39,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.35,"48.8410481577,2.25944805867","Host Has Profile Pic,Is Location Exact",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40023,2072.0,4,Ménilmontant,2,1.0,1.0,1.0,1,19,40.0,300.0,19.0,1,5,7.0,10,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.71,"48.8619682733,2.40198612937","Host Has Profile Pic,Is Location Exact,Instant...",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,0,1,1,1,1,0,0,1,1,0,1
40024,1983.0,4,Ménilmontant,2,1.0,0.0,2.0,1,15,26.0,200.0,25.0,1,0,240.0,0,80.0,10.0,10.0,10.0,10.0,10.0,10.0,0.13,"48.8664429191,2.39643395508","Host Has Profile Pic,Host Identity Verified,Is...",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0
40025,1863.0,3,Ménilmontant,2,1.0,1.0,1.0,0,15,50.0,0.0,0.0,1,0,7.0,10,100.0,10.0,10.0,9.0,10.0,10.0,10.0,0.55,"48.8663673103,2.39473441637","Host Has Profile Pic,Is Location Exact",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,1,0,0
40026,2674.0,5,Ménilmontant,2,1.0,1.0,1.0,1,10,40.0,200.0,0.0,1,30,0.0,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.47,"48.8680706349,2.39800286126","Host Has Profile Pic,Host Identity Verified,Is...",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1,1,0,1,1,1,0


In [294]:
df9.drop(columns=['Features'], inplace=True)

# Colonne 'Review Score'

On ne conserve que la colonne 'Review Score Rating' qui correspond à la moyenne pondérée de tous les autres scores sur comment s'est globalement passé le séjour pour éviter des problèmes de colinéarité entre les différentes variables de scores

In [295]:
df10 = df9.copy()

In [296]:
df10.columns

Index(['Host Since', 'Host Verifications', 'Neighbourhood Cleansed',
       'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type',
       'Amenities', 'Price', 'Security Deposit', 'Cleaning Fee',
       'Guests Included', 'Extra People', 'Calendar Updated',
       'Availability 60', 'Review Scores Rating', 'Review Scores Accuracy',
       'Review Scores Cleanliness', 'Review Scores Checkin',
       'Review Scores Communication', 'Review Scores Location',
       'Review Scores Value', 'Reviews per Month', 'Geolocation',
       'strict_cancel_policy', 'moderate_cancel_policy',
       'flexible_cancel_policy', 'super_strict_cancel_policy', 'entire_home',
       'private_room', 'shared_room', 'property_hostel',
       'property_bed_breakfast', 'property_apartment', 'property_house',
       'other_property', 'Heating', 'Kitchen', 'Wireless Internet',
       'Essentials', 'Washer', 'TV', 'Family/kid friendly', 'Hair dryer',
       'Iron', 'Elevator in building', 'Dryer', '24-hour check

In [297]:
df10.drop(columns=['Review Scores Accuracy','Review Scores Cleanliness', 'Review Scores Checkin',
                   'Review Scores Communication', 'Review Scores Location','Review Scores Value'], 
          inplace=True)

In [298]:
df10.head()

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Reviews per Month,Geolocation,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property,Heating,Kitchen,Wireless Internet,Essentials,Washer,TV,Family/kid friendly,Hair dryer,Iron,Elevator in building,Dryer,24-hour check-in,Smoking allowed,Breakfast,Pets allowed,Wheelchair accessible,Air conditioning,Free parking on premises,host_reviews_verified,host_phone_verified,host_email_verified,host_government_id_verified,Host Is Superhost,Is Location Exact,Host Has Profile Pic,Host Identity Verified,Instant Bookable
0,1884.0,3,Passy,2,1.0,1.0,1.0,1,14,50.0,500.0,30.0,1,0,4.0,15,80.0,0.16,"48.8445603313,2.26336692547",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0
1,1434.0,3,Passy,5,2.0,2.0,2.0,1,23,250.0,750.0,90.0,4,15,0.0,28,95.0,0.56,"48.8580714183,2.28022090415",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,0,0,1,1,0,0
2,1268.0,4,Passy,7,1.5,2.0,5.0,1,19,165.0,200.0,30.0,1,0,0.0,51,100.0,1.00,"48.8502032736,2.27083162889",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,1
3,1587.0,3,Passy,2,1.0,0.0,1.0,1,13,35.0,94.0,0.0,1,0,120.0,0,93.0,1.38,"48.8420130083,2.26885375663",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0
4,2269.0,3,Passy,4,1.0,1.0,2.0,1,15,90.0,500.0,30.0,2,10,180.0,39,100.0,0.35,"48.8410481577,2.25944805867",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0


# Colonne 'POI'

Enrichissement de notre Dataset en ajoutant pour chaque annonce, la distance moyenne entre l'appartement et les 30 plus proches points d'intérêt dans Paris. 

In [299]:
df11 = df10.copy()

## Calcul des distances entre les appartements et les lieux touristiques

### Import du DataFrame sur les points d'intérêt

In [301]:
df_tourisme = pd.read_csv("paris_poi.csv", index_col=0)

In [302]:
df_tourisme

,address,lat,lon,name,subCategory
0,23 Quai Anatole France,48.862078,2.322534,Concorde Atlantique,Boat or Ferry
1,17 Place du Trocadéro,48.862301,2.288225,Palais de Chaillot,Monument / Landmark
2,Quai François Mauriac,48.833679,2.375686,Bibliothèque Nationale de France (BNF),Library
3,Place Charles de Gaulle,48.873797,2.294952,Arc de Triomphe,Monument / Landmark
4,4 Boulevard du Palais,48.855920,2.344637,Palais de Justice de Paris,Courthouse
...,...,...,...,...,...
1729,159 Boulevard Malesherbe,48.885632,2.306501,Cabinet Dentaire - Dr Cadet & Dr Bertrant,Dentist's Office
1730,10 Rue Civiale,48.871437,2.374117,cabinet paramedical civiale,Medical Center
1731,Rue Pajol,48.866977,2.353663,Bibliothèque Vaclav Havel,Library
1732,10 Rue Louis-Blanc,48.879165,2.369245,Consulat Chaoui - Paris X,Embassy / Consulate


### Isolation des latitudes et longitudes dans le DataFrame Airbnb

In [303]:
df11['lat'] = df11.Geolocation.apply(lambda x : float(x.split(',')[0]))

In [304]:
df11['lon'] = df11.Geolocation.apply(lambda x : float(x.split(',')[1]))

### Calcul des distances moyennes

Valeur de conversion géodésique -> km : 77.14017896147689 km/geod  
Obtenu depuis google maps à Paris

In [307]:
def nearest_average(df_, i, df_t, n:int):
    
    """
    return the index in df_t of the "n" neareast and their distances
    in the form (distances, indices)
    """
    x = np.array([df_['lat'][i], df_['lon'][i]])
    ys = np.array(list(zip(df_t['lat'],  df_t['lon'])))
    #distances in km
    distances = np.linalg.norm(x - ys, axis=1) * 77.14017896147689
    indices = distances.argsort()[:n]
    
    return np.mean(distances[indices])

In [308]:
%%time
df11["30 nearest"] = [nearest_average(df11, i, df_tourisme, 30) for i in range(len(df11))]

CPU times: user 40.2 s, sys: 34.5 ms, total: 40.3 s
Wall time: 40.3 s


In [309]:
df11

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Reviews per Month,Geolocation,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property,Heating,Kitchen,Wireless Internet,Essentials,Washer,TV,Family/kid friendly,Hair dryer,Iron,Elevator in building,Dryer,24-hour check-in,Smoking allowed,Breakfast,Pets allowed,Wheelchair accessible,Air conditioning,Free parking on premises,host_reviews_verified,host_phone_verified,host_email_verified,host_government_id_verified,Host Is Superhost,Is Location Exact,Host Has Profile Pic,Host Identity Verified,Instant Bookable,lat,lon,30 nearest
0,1884.0,3,Passy,2,1.0,1.0,1.0,1,14,50.0,500.0,30.0,1,0,4.0,15,80.0,0.16,"48.8445603313,2.26336692547",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.844560,2.263367,0.257026
1,1434.0,3,Passy,5,2.0,2.0,2.0,1,23,250.0,750.0,90.0,4,15,0.0,28,95.0,0.56,"48.8580714183,2.28022090415",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,0,0,1,1,0,0,48.858071,2.280221,0.332669
2,1268.0,4,Passy,7,1.5,2.0,5.0,1,19,165.0,200.0,30.0,1,0,0.0,51,100.0,1.00,"48.8502032736,2.27083162889",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,1,48.850203,2.270832,0.224522
3,1587.0,3,Passy,2,1.0,0.0,1.0,1,13,35.0,94.0,0.0,1,0,120.0,0,93.0,1.38,"48.8420130083,2.26885375663",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.842013,2.268854,0.373449
4,2269.0,3,Passy,4,1.0,1.0,2.0,1,15,90.0,500.0,30.0,2,10,180.0,39,100.0,0.35,"48.8410481577,2.25944805867",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.841048,2.259448,0.343726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40023,2072.0,4,Ménilmontant,2,1.0,1.0,1.0,1,19,40.0,300.0,19.0,1,5,7.0,10,100.0,0.71,"48.8619682733,2.40198612937",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,0,1,1,1,1,0,0,1,1,0,1,48.861968,2.401986,0.348375
40024,1983.0,4,Ménilmontant,2,1.0,0.0,2.0,1,15,26.0,200.0,25.0,1,0,240.0,0,80.0,0.13,"48.8664429191,2.39643395508",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0,48.866443,2.396434,0.275906
40025,1863.0,3,Ménilmontant,2,1.0,1.0,1.0,0,15,50.0,0.0,0.0,1,0,7.0,10,100.0,0.55,"48.8663673103,2.39473441637",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.866367,2.394734,0.354959
40026,2674.0,5,Ménilmontant,2,1.0,1.0,1.0,1,10,40.0,200.0,0.0,1,30,0.0,0,98.0,0.47,"48.8680706349,2.39800286126",0,1,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1,1,0,1,1,1,0,48.868071,2.398003,0.268313


In [310]:
df11["30 nearest"].describe()

count    40028.000000
mean         0.409778
std          0.069755
min          0.183167
25%          0.368352
50%          0.412295
75%          0.449757
max          2.117988
Name: 30 nearest, dtype: float64

# Colonne 'prix_loyer_arr'

Pour modéliser les différences de prix entre les arrondissements => création d'une nouvelle variable incluant le prix moyen au m2 d'une location dans le quartier correspondant.  
Sources des prix moyen au m2 = SeLoger.com

In [311]:
df12 = df11.copy()

In [312]:
df12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40028 entries, 0 to 40027
Data columns (total 61 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Host Since                   40026 non-null  float64
 1   Host Verifications           40028 non-null  int64  
 2   Neighbourhood Cleansed       40028 non-null  object 
 3   Accommodates                 40028 non-null  int64  
 4   Bathrooms                    40028 non-null  float64
 5   Bedrooms                     40028 non-null  float64
 6   Beds                         40028 non-null  float64
 7   Bed Type                     40028 non-null  int64  
 8   Amenities                    40028 non-null  int64  
 9   Price                        40028 non-null  float64
 10  Security Deposit             40028 non-null  float64
 11  Cleaning Fee                 40028 non-null  float64
 12  Guests Included              40028 non-null  int64  
 13  Extra People    

In [313]:
df12 = df12.dropna()

In [330]:
df12['Neighbourhood Cleansed'].value_counts()

Buttes-Montmartre      4418
Popincourt             4026
Entrepôt               2869
Vaugirard              2845
Batignolles-Monceau    2355
Ménilmontant           2066
Buttes-Chaumont        2011
Temple                 1996
Passy                  1836
Opéra                  1778
Hôtel-de-Ville         1608
Panthéon               1584
Observatoire           1559
Reuilly                1528
Luxembourg             1456
Gobelins               1384
Bourse                 1373
Palais-Bourbon         1308
Élysée                 1060
Louvre                  931
Name: Neighbourhood Cleansed, dtype: int64

In [332]:
def ar_loyer(x):
    ar_loyer_m2 = {
    "Louvre": 37.1, "Bourse": 36.3, "Temple": 37.3, "Hôtel-de-Ville": 37.0, "Panthéon": 35.3, "Luxembourg": 38.4,
    "Palais-Bourbon": 36.9, "Élysée": 34.6, "Opéra": 33.3, "Entrepôt": 32.7, "Popincourt": 32.2, "Reuilly": 29.2,
    "Gobelins": 29.0, "Observatoire": 30.5, "Vaugirard": 30.5, "Passy": 32.6, "Batignolles-Monceau": 32.2, 
    "Buttes-Montmartre": 30.9, "Buttes-Chaumont": 27.4, "Ménilmontant": 28.0
    }
    try:
        ar_loyer_m2[x]
    except:
        print(x)
    return ar_loyer_m2[x]

In [333]:
df12['prix_loyer_arr'] = df12['Neighbourhood Cleansed'].apply(lambda x :ar_loyer(str(x)))

# Export du dataset final

In [336]:
df13 = df12.copy()

In [337]:
df13.head()

,Host Since,Host Verifications,Neighbourhood Cleansed,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Calendar Updated,Availability 60,Review Scores Rating,Reviews per Month,Geolocation,strict_cancel_policy,moderate_cancel_policy,flexible_cancel_policy,super_strict_cancel_policy,entire_home,private_room,shared_room,property_hostel,property_bed_breakfast,property_apartment,property_house,other_property,Heating,Kitchen,Wireless Internet,Essentials,Washer,TV,Family/kid friendly,Hair dryer,Iron,Elevator in building,Dryer,24-hour check-in,Smoking allowed,Breakfast,Pets allowed,Wheelchair accessible,Air conditioning,Free parking on premises,host_reviews_verified,host_phone_verified,host_email_verified,host_government_id_verified,Host Is Superhost,Is Location Exact,Host Has Profile Pic,Host Identity Verified,Instant Bookable,lat,lon,30 nearest,prix_loyer_arr
0,1884.0,3,Passy,2,1.0,1.0,1.0,1,14,50.0,500.0,30.0,1,0,4.0,15,80.0,0.16,"48.8445603313,2.26336692547",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.844560,2.263367,0.257026,32.6
1,1434.0,3,Passy,5,2.0,2.0,2.0,1,23,250.0,750.0,90.0,4,15,0.0,28,95.0,0.56,"48.8580714183,2.28022090415",1,0,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1,1,0,0,1,1,0,0,48.858071,2.280221,0.332669,32.6
2,1268.0,4,Passy,7,1.5,2.0,5.0,1,19,165.0,200.0,30.0,1,0,0.0,51,100.0,1.00,"48.8502032736,2.27083162889",0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,1,48.850203,2.270832,0.224522,32.6
3,1587.0,3,Passy,2,1.0,0.0,1.0,1,13,35.0,94.0,0.0,1,0,120.0,0,93.0,1.38,"48.8420130083,2.26885375663",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.842013,2.268854,0.373449,32.6
4,2269.0,3,Passy,4,1.0,1.0,2.0,1,15,90.0,500.0,30.0,2,10,180.0,39,100.0,0.35,"48.8410481577,2.25944805867",0,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,48.841048,2.259448,0.343726,32.6


In [338]:
df13.drop(columns=['Neighbourhood Cleansed','Geolocation','lat','lon'], inplace=True)

In [339]:
df13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39991 entries, 0 to 40027
Data columns (total 58 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Host Since                   39991 non-null  float64
 1   Host Verifications           39991 non-null  int64  
 2   Accommodates                 39991 non-null  int64  
 3   Bathrooms                    39991 non-null  float64
 4   Bedrooms                     39991 non-null  float64
 5   Beds                         39991 non-null  float64
 6   Bed Type                     39991 non-null  int64  
 7   Amenities                    39991 non-null  int64  
 8   Price                        39991 non-null  float64
 9   Security Deposit             39991 non-null  float64
 10  Cleaning Fee                 39991 non-null  float64
 11  Guests Included              39991 non-null  int64  
 12  Extra People                 39991 non-null  int64  
 13  Calendar Updated

In [341]:
df13.to_csv("final_cleaned_dataset.csv", index=False)